In [45]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report,accuracy_score
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import svm
import os
from keras.preprocessing import image
import cv2
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

ERROR! Session/line number was not unique in database. History logging moved to new session 186


Functions used for data pre-processing

In [46]:
def extract_features_labels(task_type):
    """
    This funtion extracts the features for all images in the folder 'dataset/..'.
    It also extracts the tumor label for each image.
    :return:
        pic_features:  an array containing 68 landmark points for each image in which a face was detected
        tumor_labels:      an array containing the gender label (male=0 and female=1) for each image in
                            which a face was detected
    """

    all_features = []
    all_labels = []
    
    image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
#     print(image_paths)   # ‘.\\assignment_dataset\\image\\IMAGE_0032.jpg'
    labels_file = open(os.path.join(basedir, labels_filename), 'r')

    lines_of_label = labels_file.readlines()
#     print(lines_of_label)   # 'IMAGE_0000.jpg,meningioma_tumor,,脑膜瘤\n'

    type_labels = {line.split(',')[0] : line.split(',')[1] for line in lines_of_label[1:]}
    
    if task_type == "Binary":
    
        # convert tumor names to int
        for k,v in type_labels.items():
            if v == "meningioma_tumor\n":
                type_labels[k] = 1
            elif v == 'glioma_tumor\n':
                type_labels[k] = 1
            elif v == 'pituitary_tumor\n':
                type_labels[k] = 1
            elif v == 'no_tumor\n':
                type_labels[k] = 0
                
    elif task_type == "Multiclass":
        for k,v in type_labels.items():
            if v == "meningioma_tumor\n":
                type_labels[k] = 1
            elif v == 'glioma_tumor\n':
                type_labels[k] = 2
            elif v == 'pituitary_tumor\n':
                type_labels[k] = 3
            elif v == 'no_tumor\n':
                type_labels[k] = 0

    
    for img in image_paths:
        # pic_features
        img_read = cv2.imread(img, flags = 0)  # 把图片转化成 numpy.ndarray，shape=(512, 512)
        img_read = cv2.pyrDown(img_read) 
        img_read = cv2.pyrDown(img_read) 
        all_features.append(img_read)   # list
        
        # tumor_labels
        file_name= img.split('\\')[-1]
        all_labels.append(type_labels[file_name])

    pic_features = np.array(all_features)
    tumor_labels = np.array(all_labels) 
    print("step 1 finish")

    return pic_features, tumor_labels

def get_data(task_type):

    X,y = extract_features_labels(task_type)  
    print(X.shape)
    print(y.shape)
    # X: landmark_features - ndarray, shape = (134, 68, 2)
    # y: gender_labels - list, len= 134
    
    
    Y = np.array([y, -(y - 1)]).T   # Y: ndarray, shape = (134, 2)
    print(Y.shape)
    tr_X = X[:2500]
    tr_Y = Y[:2500]
    te_X = X[2500:]
    te_Y = Y[2500:]
    
    tr_X2 = tr_X.reshape((2500, 128*128))
    tr_Y2 = list(zip(*tr_Y))[0]
    te_X2 = te_X.reshape((500, 128*128))
    te_Y2 = list(zip(*te_Y))[0]
    
    print()
    print("tr_X2:")
    print(tr_X2)
    print(tr_X2.shape)   # (100, 512*512)
    print(type(tr_X2))   # numpy.ndarray
    print()
    print("tr_Y2:")
#     print(tr_Y2)
    print(type(tr_Y2))  # tuple
    print(len(tr_Y2))    # len=100
    print()
    print("te_X2:")
    print(te_X2.shape)   # (56, 512*512)
    print(type(te_X2))   # numpy.ndarray
    print()
    print("te_Y2:")
    print(te_Y2)
    print(type(te_Y2))  # tuple
    print(len(te_Y2))    # len=56

    return tr_X2, tr_Y2, te_X2, te_Y2

Task A - SVM：in this section, use SVM algorithm to train models for Binary task 

In [47]:
basedir = '.\\assignment_dataset'
images_dir = os.path.join(basedir,'image')
labels_filename = 'label.csv'
# print(images_dir)   # .\assignment_dataset\image


def img_SVM(training_images, training_labels, test_images, test_labels):
    classifier = svm.SVC(kernel='linear')

    classifier.fit(training_images, training_labels)

    pred = classifier.predict(test_images)

    print(pred)

    print("Accuracy:", accuracy_score(test_labels, pred))
    return pred


tr_X, tr_Y, te_X, te_Y= get_data(task_type = "Binary")
pred=img_SVM(tr_X, tr_Y, te_X, te_Y)
# 喂给机器的数据： 100个压缩成2维的array，长度为100的包含gender信息的元组，34个x_test，34个y_test            

# Using 500 validation set to test accuracy



step 1 finish
(3000, 128, 128)
(3000,)
(3000, 2)

tr_X2:
[[ 0  0  0 ...  2  1  1]
 [20 22 23 ...  1  3 22]
 [ 9  4  2 ...  0  0  1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 6  5  4 ...  3  3  3]
 [ 1  1  1 ...  2  2  2]]
(2500, 16384)
<class 'numpy.ndarray'>

tr_Y2:
<class 'tuple'>
2500

te_X2:
(500, 16384)
<class 'numpy.ndarray'>

te_Y2:
(1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 

Task A - SVM testing：using test dataset given to calculate accuracy

In [35]:
basedir = '.\\test'
images_dir = os.path.join(basedir,'image')
labels_filename = 'label.csv'

def get_testing_data(task_type):

    X,y = extract_features_labels(task_type)   
    # X: landmark_features - ndarray, shape = (134, 68, 2)
    # y: gender_labels - list, len= 134
    
    
    Y = np.array([y, -(y - 1)]).T   # Y: ndarray, shape = (200, 2)
    print(1111111111111)
    print(Y.shape)

    testing_X = X.reshape((200, 128*128))
    testing_Y = list(zip(*Y))[0]

    print("testing_X:")
    print(testing_X.shape)   # (56, 512*512)
    print(type(testing_X))   # numpy.ndarray
    print()
    print("testing_Y:")
    print(testing_Y)
    print(type(testing_Y))  # tuple
    print(len(testing_Y))    # len=56

    return testing_X, testing_Y

def img_SVM_testing(training_images, training_labels, test_images, test_labels):
    classifier = svm.SVC(kernel='linear')

    classifier.fit(training_images, training_labels)

    pred_testing = classifier.predict(test_images)

    print(pred_testing)

    print("Test Accuracy:", accuracy_score(test_labels, pred_testing))
    return pred_testing


testing_X, testing_Y= get_testing_data(task_type="Binary")
pred_testing=img_SVM_testing(tr_X, tr_Y, testing_X, testing_Y)
# 喂给机器的数据： 100个压缩成2维的array，长度为100的包含gender信息的元组，34个x_test，34个y_test 

1111111111111
(200, 2)
testing_X:
(200, 16384)
<class 'numpy.ndarray'>

testing_Y:
(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1)
<class 'tuple'>
200
[1 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 0 1
 1 1 1 0 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0
 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 

Task B - SVM：in this section, use SVM algorithm to train models for Multiclass task 

In [36]:
basedir = '.\\assignment_dataset'
images_dir = os.path.join(basedir,'image')
labels_filename = 'label.csv'
# print(images_dir)   # .\assignment_dataset\image


def img_SVM(training_images, training_labels, test_images, test_labels):
    classifier = svm.SVC(kernel='linear')

    classifier.fit(training_images, training_labels)

    pred = classifier.predict(test_images)

    print(pred)

    print("Accuracy:", accuracy_score(test_labels, pred))


tr_X, tr_Y, te_X, te_Y= get_data(task_type = "Multiclass")
pred=img_SVM(tr_X, tr_Y, te_X, te_Y)
# 喂给机器的数据： 100个压缩成2维的array，长度为100的包含gender信息的元组，34个x_test，34个y_test            
            



(3000, 2)

tr_X2:
[[ 0  0  0 ...  2  1  1]
 [20 22 23 ...  1  3 22]
 [ 9  4  2 ...  0  0  1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 6  5  4 ...  3  3  3]
 [ 1  1  1 ...  2  2  2]]
(2500, 16384)
<class 'numpy.ndarray'>

tr_Y2:
<class 'tuple'>
2500

te_X2:
(500, 16384)
<class 'numpy.ndarray'>

te_Y2:
(2, 0, 3, 1, 1, 3, 3, 2, 1, 1, 0, 0, 1, 1, 3, 2, 0, 3, 2, 3, 1, 2, 2, 0, 3, 1, 1, 2, 1, 1, 2, 2, 0, 3, 1, 0, 2, 1, 0, 1, 3, 2, 2, 3, 2, 1, 3, 2, 1, 2, 0, 1, 1, 1, 3, 0, 2, 3, 1, 3, 2, 0, 1, 3, 1, 1, 3, 2, 2, 0, 2, 3, 1, 2, 1, 1, 0, 3, 2, 2, 3, 0, 2, 1, 2, 2, 1, 1, 3, 2, 3, 3, 3, 3, 1, 1, 2, 1, 3, 0, 1, 2, 2, 3, 1, 2, 3, 0, 2, 3, 2, 1, 2, 3, 2, 0, 1, 3, 1, 3, 1, 3, 1, 3, 1, 0, 2, 2, 3, 3, 3, 0, 3, 0, 0, 1, 3, 0, 2, 2, 3, 2, 1, 1, 0, 3, 1, 3, 1, 2, 3, 1, 1, 3, 0, 1, 0, 1, 3, 2, 3, 2, 3, 3, 1, 3, 2, 2, 3, 1, 1, 1, 3, 2, 1, 2, 3, 2, 2, 3, 1, 2, 1, 3, 2, 0, 2, 2, 2, 1, 2, 0, 3, 2, 2, 3, 3, 0, 1, 0, 1, 2, 1, 3, 3, 3, 2, 1, 0, 3, 0, 3, 2, 1, 0, 2, 3, 3, 1, 1, 1, 2, 1, 3, 2, 1, 1, 2, 3, 0, 2, 1, 2, 2, 1, 

Task B - SVM testing：using test dataset given to calculate accuracy

In [37]:
basedir = '.\\test'
images_dir = os.path.join(basedir,'image')
labels_filename = 'label.csv'

def get_testing_data(task_type):

    X,y = extract_features_labels(task_type)   
    # X: landmark_features - ndarray, shape = (134, 68, 2)
    # y: gender_labels - list, len= 134
    
    
    Y = np.array([y, -(y - 1)]).T   # Y: ndarray, shape = (200, 2)
    print(1111111111111)
    print(Y.shape)

    testing_X = X.reshape((200, 128*128))
    testing_Y = list(zip(*Y))[0]

    print("testing_X:")
    print(testing_X.shape)   # (56, 512*512)
    print(type(testing_X))   # numpy.ndarray
    print()
    print("testing_Y:")
    print(testing_Y)
    print(type(testing_Y))  # tuple
    print(len(testing_Y))    # len=56

    return testing_X, testing_Y

def img_SVM_testing(training_images, training_labels, test_images, test_labels):
    classifier = svm.SVC(kernel='linear')

    classifier.fit(training_images, training_labels)

    pred_testing = classifier.predict(test_images)

    print(pred_testing)

    print("Test Accuracy:", accuracy_score(test_labels, pred_testing))
    return pred_testing


testing_X, testing_Y= get_testing_data(task_type="Multiclass")
pred_testing=img_SVM_testing(tr_X, tr_Y, testing_X, testing_Y)
# 喂给机器的数据： 100个压缩成2维的array，长度为100的包含gender信息的元组，34个x_test，34个y_test 

1111111111111
(200, 2)
testing_X:
(200, 16384)
<class 'numpy.ndarray'>

testing_Y:
(1, 1, 2, 3, 2, 3, 3, 3, 3, 2, 0, 0, 3, 1, 2, 3, 3, 1, 1, 1, 3, 1, 1, 1, 2, 3, 0, 1, 0, 0, 0, 2, 1, 1, 1, 2, 2, 3, 1, 3, 0, 0, 0, 3, 1, 3, 3, 1, 1, 0, 0, 3, 2, 2, 3, 1, 2, 1, 3, 2, 1, 1, 1, 2, 2, 0, 3, 3, 3, 1, 2, 3, 1, 2, 3, 2, 0, 1, 3, 2, 0, 2, 3, 1, 0, 3, 2, 3, 1, 1, 3, 2, 1, 3, 1, 2, 2, 0, 3, 3, 0, 0, 0, 0, 3, 0, 1, 0, 1, 3, 2, 3, 1, 1, 0, 2, 1, 1, 1, 0, 1, 1, 3, 1, 1, 3, 1, 3, 3, 3, 2, 3, 3, 3, 1, 1, 2, 0, 0, 2, 3, 3, 2, 0, 0, 1, 1, 1, 1, 1, 1, 0, 2, 0, 0, 2, 1, 1, 0, 2, 2, 1, 3, 3, 2, 1, 2, 2, 1, 2, 1, 2, 1, 3, 2, 1, 2, 3, 1, 2, 3, 3, 2, 0, 1, 3, 1, 1, 1, 1, 1, 1, 0, 1, 0, 3, 0, 1, 0, 2)
<class 'tuple'>
200
[3 1 2 3 1 3 3 3 3 0 0 0 3 1 2 3 3 1 0 1 3 1 1 1 2 3 0 1 0 3 0 1 1 1 0 2 2
 3 1 3 0 0 0 3 1 3 3 1 1 0 0 3 1 2 3 3 2 2 3 2 2 1 3 2 2 0 3 3 3 1 2 3 2 0
 3 2 1 1 3 2 0 2 3 1 0 3 2 3 1 1 3 2 1 3 1 2 2 0 2 3 0 1 0 0 3 0 2 0 1 3 2
 3 1 1 1 2 1 1 1 0 3 1 3 1 1 3 1 3 3 3 2 1 3 3 2 1 1 1 0 1 3 3 2 0 0 1 

in this section, use different kernel functions and plot the support vectors

Task A - SVM：in this section, use KNN algorithm to train models for Binary task

In [48]:
k = 4
def KNNClassifier(tr_X, tr_Y, te_X,k):

    #Create KNN object with a K coefficient
    neigh = KNeighborsClassifier(n_neighbors=k)   
    # KNeighborsClassifier 是自带的一个函数
    # 其中的一个参数 n_neighbors：k neighbors查询使用的邻居数。就是k-NN的k的值，选取最近的k个点
    neigh.fit(tr_X, tr_Y) # Fit KNN model


    Y_pred = neigh.predict(te_X)
    return Y_pred
print(11111111111111)
tr_X, tr_Y, te_X, te_Y= get_data(task_type = "Binary")
print(tr_X.shape)
print(len(tr_Y))
Y_pred=KNNClassifier(tr_X, tr_Y, te_X,k)
print(Y_pred)
score=metrics.accuracy_score(te_Y,Y_pred)
print(score)

11111111111111
step 1 finish
(3000, 128, 128)
(3000,)
(3000, 2)

tr_X2:
[[ 0  0  0 ...  2  1  1]
 [20 22 23 ...  1  3 22]
 [ 9  4  2 ...  0  0  1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 6  5  4 ...  3  3  3]
 [ 1  1  1 ...  2  2  2]]
(2500, 16384)
<class 'numpy.ndarray'>

tr_Y2:
<class 'tuple'>
2500

te_X2:
(500, 16384)
<class 'numpy.ndarray'>

te_Y2:
(1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 

Task B - SVM：in this section, use KNN algorithm to train models for Multiclass task

In [49]:
k = 4
def KNNClassifier(tr_X, tr_Y, te_X,k):

    #Create KNN object with a K coefficient
    neigh = KNeighborsClassifier(n_neighbors=k)   
    # KNeighborsClassifier 是自带的一个函数
    # 其中的一个参数 n_neighbors：k neighbors查询使用的邻居数。就是k-NN的k的值，选取最近的k个点
    neigh.fit(tr_X, tr_Y) # Fit KNN model


    Y_pred = neigh.predict(te_X)
    return Y_pred

tr_X, tr_Y, te_X, te_Y= get_data(task_type = "Multiclass")
Y_pred=KNNClassifier(tr_X, tr_Y, te_X,k)
print(Y_pred)
score=metrics.accuracy_score(te_Y,Y_pred)
print(score)

step 1 finish
(3000, 128, 128)
(3000,)
(3000, 2)

tr_X2:
[[ 0  0  0 ...  2  1  1]
 [20 22 23 ...  1  3 22]
 [ 9  4  2 ...  0  0  1]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 6  5  4 ...  3  3  3]
 [ 1  1  1 ...  2  2  2]]
(2500, 16384)
<class 'numpy.ndarray'>

tr_Y2:
<class 'tuple'>
2500

te_X2:
(500, 16384)
<class 'numpy.ndarray'>

te_Y2:
(2, 0, 3, 1, 1, 3, 3, 2, 1, 1, 0, 0, 1, 1, 3, 2, 0, 3, 2, 3, 1, 2, 2, 0, 3, 1, 1, 2, 1, 1, 2, 2, 0, 3, 1, 0, 2, 1, 0, 1, 3, 2, 2, 3, 2, 1, 3, 2, 1, 2, 0, 1, 1, 1, 3, 0, 2, 3, 1, 3, 2, 0, 1, 3, 1, 1, 3, 2, 2, 0, 2, 3, 1, 2, 1, 1, 0, 3, 2, 2, 3, 0, 2, 1, 2, 2, 1, 1, 3, 2, 3, 3, 3, 3, 1, 1, 2, 1, 3, 0, 1, 2, 2, 3, 1, 2, 3, 0, 2, 3, 2, 1, 2, 3, 2, 0, 1, 3, 1, 3, 1, 3, 1, 3, 1, 0, 2, 2, 3, 3, 3, 0, 3, 0, 0, 1, 3, 0, 2, 2, 3, 2, 1, 1, 0, 3, 1, 3, 1, 2, 3, 1, 1, 3, 0, 1, 0, 1, 3, 2, 3, 2, 3, 3, 1, 3, 2, 2, 3, 1, 1, 1, 3, 2, 1, 2, 3, 2, 2, 3, 1, 2, 1, 3, 2, 0, 2, 2, 2, 1, 2, 0, 3, 2, 2, 3, 3, 0, 1, 0, 1, 2, 1, 3, 3, 3, 2, 1, 0, 3, 0, 3, 2, 1, 0, 2, 3, 3, 1, 1, 1, 2, 

Plot the relationship between k value and test accuracy for k values in range [0,20] using the matplotlib library.

In [ ]:
score_list=[]
print(score_list)

for i in range(1,20):
    print(i)
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(tr_X,tr_Y)
    pred_i = knn.predict(te_X)
    score_list.append(metrics.accuracy_score(te_Y,pred_i))
    print(score_list)  
print(score_list)    

In [ ]:
plt.plot(range(1,36),score_list,color='pink', linestyle='dashed', marker='o', markerfacecolor='grey',markersize=10)
plt.title("Accuracy vs. K Value")
plt.xlabel("K")
plt.ylabel("Accuracy")
plt.show()